# Please make sure the right path of the adversaial files. Specially the last part

In [ ]:
image_path = "/content/drive/MyDrive/Subset_1-40_81-120/Image_subset_032224"
FGSM_path = "/content/drive/MyDrive/Subset_1-40_81-120/Adversarial_Examples/FGSM/"

In [ ]:
!pip install adversarial-robustness-toolbox

In [ ]:
import tensorflow as tf

tf.compat.v1.disable_eager_execution()

In [ ]:
tf.config.list_physical_devices('GPU')

[]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
## load package
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from keras.models import Sequential
from sklearn.model_selection import train_test_split

In [ ]:
from tensorflow.keras.applications import EfficientNetV2B0 as Model

model = Model(weights='imagenet', include_top=True)
model.compile(
        loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adam(lr=0.01), metrics=["accuracy"]
    )
model.summary()

Model: "efficientnetv2-b0"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 rescaling_1 (Rescaling)     (None, 224, 224, 3)          0         ['input_2[0][0]']             
                                                                                                  
 normalization_1 (Normaliza  (None, 224, 224, 3)          0         ['rescaling_1[0][0]']         
 tion)                                                                                            
                                                                                                  
 stem_conv (Conv2D)          (None, 112, 112, 32)         864       ['normalizatio

In [ ]:
from art.attacks.evasion import FastGradientMethod
from art.estimators.classification import KerasClassifier
classifier = KerasClassifier(model, clip_values=(0, 255), use_logits=False)
FGSM = FastGradientMethod(classifier, eps= .005*255)

In [ ]:
from PIL import Image

def change_size_for_model(image):
    image = Image.fromarray(image)
    resized_image = image.resize((224, 224))
    resized_array = np.array(resized_image)
    return resized_array.astype(np.uint8)

def change_size_for_original(image, original_shape):
    image = Image.fromarray(image)
    resized_image = image.resize(original_shape) #resize image
    resized_array = np.array(resized_image)
    return resized_array.astype(np.uint8)

In [ ]:
import os

In [ ]:
start_idx =0
end_idx = 41

list_of_dir =list()
for i in range(start_idx,end_idx):
  list_of_dir.append(str(i))


start_idx =81
end_idx = 121

for i in range(start_idx,end_idx):
  list_of_dir.append(str(i))


In [ ]:
for j in range(len(list_of_dir)):
# for j in range(5):
  list_of_names = os.listdir(image_path+'/'+str(list_of_dir[j]))
  list_of_names.sort()
  original_image_array = list()

  for img_name in list_of_names:
    # Load the image
    image = Image.open(image_path+'/'+str(list_of_dir[j])+'/'+img_name)

    # Convert the image to a numpy array
    original_image_array.append(np.array(image))

  resized_image_array = list()
  for img in original_image_array:
    resized_image_array.append(change_size_for_model(img))

  resized_image_array = np.array(resized_image_array)

  labels = np.zeros((resized_image_array.shape[0], 1000))
  labels[:, int(list_of_dir[j])] = 1
  new_img_fgsm= FGSM.generate(resized_image_array.astype(np.float32), labels)

  folder_for_adv = FGSM_path + str(list_of_dir[j])
  if not os.path.exists(folder_for_adv):
      # The folder does not exist, so create it
      os.makedirs(folder_for_adv, exist_ok=True)
      print(f"Folder '{folder_for_adv}' was created.")
  else:
      # The folder already exists
      print(f"Folder '{folder_for_adv}' already exists.")

  for i in range(new_img_fgsm.shape[0]):
    a = new_img_fgsm[i]
    a[np.where(a > 255)] = 255
    a[np.where(a < 0)] = 0
    plt.figure();
    plt.imshow(a.astype(np.uint8))
    plt.imsave(folder_for_adv+'/' +str(list_of_names[i]), a.astype(np.uint8))


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
Stop

NameError: name 'Stop' is not defined